In [2]:
import pandas as pd
import seaborn as sns

# Loading the datasets

In [4]:
wines_df = pd.read_csv("XWines_Full_21M_ratings.csv") 
display(wines_df.head(20))

#rating_df=pd.read_csv("XWines_Full_100K_wines.csv") 
#display(rating_df.head(20))

C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_22396\46053177.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  wines_df = pd.read_csv("XWines_Full_21M_ratings.csv")


,RatingID,UserID,WineID,Vintage,Rating,Date
0,1,1604441,136103,1950,4.0,2019-10-14 11:20:52
1,2,1291483,136103,1950,5.0,2019-11-28 03:36:33
2,3,1070605,104036,1950,5.0,2017-12-28 10:15:55
3,4,1080181,144864,1950,5.0,2016-06-23 02:16:22
4,5,1834379,111430,1950,5.0,2021-05-16 17:58:14
5,6,1995440,157985,1950,4.0,2016-01-06 22:14:14
6,7,1166181,101794,1950,5.0,2018-04-15 12:04:46
7,8,1839846,136103,1950,5.0,2020-07-18 15:41:19
8,9,1693747,136103,1950,1.0,2018-11-23 01:48:57
9,10,1478537,135897,1950,4.0,2015-05-04 19:52:09


In [7]:
display(wines_df[wines_df['UserID'] == 1291483]).head(3)


,RatingID,UserID,WineID,Vintage,Rating,Date
1,2,1291483,136103,1950,5.0,2019-11-28 03:36:33
4268708,4268709,1291483,167537,2011,1.0,2020-11-20 01:02:14
11269848,11269849,1291483,167428,2015,4.0,2019-11-03 01:12:49
12899046,12899047,1291483,167573,2016,4.0,2019-09-15 22:27:00
12925348,12925349,1291483,167418,2016,4.0,2019-08-15 01:13:20
14635529,14635530,1291483,167463,2017,4.5,2019-08-24 00:01:20
15056211,15056212,1291483,167433,2017,4.5,2020-04-18 00:50:22


AttributeError: 'NoneType' object has no attribute 'head'

# Merge the two dataset with wineId as the inner joint.

In [22]:
joined_df = wines_df.join(rating_df.set_index('WineID'), on='WineID', how='inner')
display(joined_df.head(1))


,RatingID,UserID,WineID,Vintage,Rating,Date,WineName,Type,Elaborate,Grapes,...,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,1,1604441,136103,1950,4.0,2019-10-14 11:20:52,Lambrusco Emilia,Red,Varietal/100%,['Lambrusco'],...,Full-bodied,High,IT,Italy,1581,Emilia,42951,Riunite,http://www.riunite.com,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."


For Grup racomander :
Aproval voting,
Multiplicative,
Feirness,
LeastMeasery,
MostPLeasure
and the other

Assume now we have a dataset which includes the  "user_id", " wine_id" and "rating", named data


In [10]:
data={
    'user_id': [1604441, 1604441, 1604441, 1291483, 1291483,1291483,1070605,1070605,1070605],
    'wine_id': [136103, 117146, 155378, 136103, 117146, 167573, 136103, 155378, 167573],
    'rating': [5, 4, 3, 4, 5, 4, 5, 3, 5]
}

In [11]:
df=pd.DataFrame(data)

# Approval Voting
theta=2.5
larger_df=df[df['rating']>=theta]
counts=larger_df['wine_id'].value_counts()
# transform into DataFrame
vote_counts=counts.reset_index()
vote_counts.columns=['wine_id','vote_count_in_total']
majority_based_recommendation=vote_counts.sort_values(by='vote_count_in_total',ascending=False)
print('Recommendation by majority based:\n',majority_based_recommendation)




Recommendation by majority based:
    wine_id  vote_count_in_total
0   136103                    3
1   117146                    2
2   155378                    2
3   167573                    2


In [12]:
# Multiplicative
grouped= df.groupby('wine_id')['rating']
def multiply_rating(ratings):
    score=1
    for rating in ratings:
        score=score*rating
    return score

final_score=grouped.apply(multiply_rating)
# transform into DataFrame
multiply_score=final_score.reset_index()
multiply_score.columns=['wine_id','final_score']
multiply_recommendation=multiply_score.sort_values(by='final_score',ascending=False)  
print('Recommendation by Multiplicative: \n',multiply_recommendation)  




Recommendation by Multiplicative: 
    wine_id  final_score
1   136103          100
0   117146           20
3   167573           20
2   155378            9


In [13]:
# Least Misery

minimal_rating=df.groupby('wine_id')['rating'].min()
minimal_rating_df=minimal_rating.reset_index()
minimal_rating_df.columns=['wine_id','minimal_rating']
least_misery_recommendation=minimal_rating_df.sort_values(by='minimal_rating',ascending=False)
print('Recommendation by Least Misery: \n',least_misery_recommendation)

Recommendation by Least Misery: 
    wine_id  minimal_rating
0   117146               4
1   136103               4
3   167573               4
2   155378               3


In [14]:
# Most Pleasure
max_rating=df.groupby('wine_id')['rating'].max()
max_rating_df=max_rating.reset_index()
max_rating_df.columns=['wine_id','maximum_rating']
most_pleasure_recommendation=max_rating_df.sort_values(by='maximum_rating',ascending=False)
print('Recommendation by Most Pleasure: \n',most_pleasure_recommendation)


Recommendation by Most Pleasure: 
    wine_id  maximum_rating
0   117146               5
1   136103               5
3   167573               5
2   155378               3
